In [1]:
import requests
import pandas as pd

/Users/apoorvashete/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/var/folders/pb/hj9rp74j56v0kk3cp85ryzkr0000gn/T/ipykernel_27091/222825045.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [36]:

def fetch_reviews_and_create_df(business_id_or_alias, api_key):
    url = f"https://api.yelp.com/v3/businesses/{business_id_or_alias}/reviews"
    headers = {"Authorization": f"Bearer {api_key}"}
    params = {"sort_by": "newest_first"}

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  

        reviews = response.json()['reviews']
        reviews_df = pd.DataFrame([{
            'business_id': business_id_or_alias,  
            'user_id': review['user']['id'],
            'name': review['user']['name'],
            'timestamp': review['time_created'],
            'rating': review['rating'],
            'text': review['text']
        } for review in reviews])

        return reviews_df

    except requests.RequestException as e:
        print(f"Request error for {business_id_or_alias}: {e}")
        return pd.DataFrame(columns=['business_id', 'user_id', 'name', 'timestamp', 'rating', 'text']) 
    
    except Exception as e:
        print(f"General error for {business_id_or_alias}: {e}")
        return pd.DataFrame(columns=['business_id', 'user_id', 'name', 'timestamp', 'rating', 'text'])  





In [9]:
# def fetch_all_reviews(business_id_or_alias, api_key):
#     url = f"https://api.yelp.com/v3/businesses/{business_id_or_alias}/reviews"
#     headers = {"Authorization": f"Bearer {api_key}"}
#     params = {"sort_by": "newest", "limit": 20}  # Adjust the limit as needed

#     all_reviews = []

#     try:
#         while True:
#             response = requests.get(url, headers=headers, params=params)
#             response.raise_for_status()

#             reviews = response.json().get('reviews', [])
#             if not reviews:
#                 break  # No more reviews

#             all_reviews.extend(reviews)

#             # Check if there are more reviews to fetch
#             if 'next' in response.links:
#                 url = response.links['next']['url']
#             else:
#                 break

#         reviews_df = pd.DataFrame([{
#             'business_id': business_id_or_alias,  
#             'user_id': review['user']['id'],
#             'name': review['user']['name'],
#             'timestamp': review['time_created'],
#             'rating': review['rating'],
#             'text': review['text']
#         } for review in all_reviews])

#         return reviews_df

#     except requests.RequestException as e:
#         print(f"Request error for {business_id_or_alias}: {e}")
#         return pd.DataFrame(columns=['business_id', 'user_id', 'name', 'timestamp', 'rating', 'text'])

#     except Exception as e:
#         print(f"General error for {business_id_or_alias}: {e}")
#         return pd.DataFrame(columns=['business_id', 'user_id', 'name', 'timestamp', 'rating', 'text'])


In [8]:

API_KEY = "yZYWXZg3aPLaumNf6XA7HxgxUpEAhHxUQA46Y2sWRuWrF413ZwMDUQal4M1ICY92_kREdURbth-wgP10_isL72XguvHfwlMejJxID6pw8e8IWsQcYBEVQ-3ALjfuZXYx"
business_id_or_alias = "P0VziPp_DuKBol_YNKLqSw"

# Fetch reviews and print the DataFrame
reviews_df = fetch_reviews_and_create_df(business_id_or_alias, API_KEY)
reviews_df

,business_id,user_id,name,timestamp,rating,text
0,P0VziPp_DuKBol_YNKLqSw,pVQ5hn7U4U2zmyHJY__cPQ,Eric A.,2024-01-29 15:45:42,5,Excellent food atmosphere and experience would...
1,P0VziPp_DuKBol_YNKLqSw,VVVAobfACVVyYHfdImnBSQ,Nuzhat B.,2024-01-25 09:42:39,5,Tiny restaurant in a mixed neighborhood of Irv...
2,P0VziPp_DuKBol_YNKLqSw,hT5o8CE_zgR5nFu3mY8iMw,Hikari N.,2024-01-16 21:56:53,5,This place transports me right back to Japan!\...


In [40]:
file_path = '/Users/sofiya/STA220FinalProject-Yelp/restaurants_data/popular_restaurants.csv'

df = pd.read_csv(file_path)

all_reviews_df = pd.DataFrame()
for restaurant_id in df['id']:
    reviews_df = fetch_reviews_and_create_df(restaurant_id, API_KEY)  # Use the global API_KEY
    if not reviews_df.empty:
        all_reviews_df = pd.concat([all_reviews_df, reviews_df], ignore_index=True)

In [41]:
output_file_path = '/Users/sofiya/STA220FinalProject-Yelp/restaurants_data/Reviews_Popular_restaurants.csv'
all_reviews_df.to_csv(output_file_path, index=False)